# Data Engineering

In [24]:
import pandas as pd
import numpy as np
import wrds

In [25]:
###################
# Connect to WRDS #
###################

db = wrds.Connection(yautoconnect=True )

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


## Linking IBES and CRSP

### 1) Create IBES CRSP Linking table

In [7]:
# 1.1 IBES: Get the list of IBES Tickers for US firms in IBES
_ibes1 = db.raw_sql("""
                        select ticker, cusip, cname, sdates
                        from ibes.id
                        where usfirm='1' 
                        and cusip != ''
                        """, date_cols=['sdates'])

In [8]:
# Create first and last 'start dates' for a given cusip

_ibes1_date = _ibes1.groupby(['ticker','cusip']).sdates.agg(['min', 'max'])\
.reset_index().rename(columns={'min':'fdate', 'max':'ldate'})

_ibes2 = pd.merge(_ibes1, _ibes1_date,how='left', on =['ticker','cusip'])
_ibes2 = _ibes2.sort_values(by=['ticker','cusip','sdates'])

# keep only the most recent company name
_ibes2 = _ibes2.loc[_ibes2.sdates == _ibes2.ldate].drop(['sdates'], axis=1)
_ibes2.head()

,ticker,cusip,cname,fdate,ldate
1,0000,87482X10,TALMER BANCORP,2014-02-20,2014-03-20
6,0001,26878510,EP ENGR CORP,2014-02-20,2019-10-17
7,0004,02504D10,AMERICAN CAPITAL,2014-02-20,2014-02-20
8,000R,14163310,CARECOM,2014-02-20,2014-02-20
9,000V,15117E10,CELLADON,2014-03-20,2014-03-20


In [9]:
# 1.2 CRSP: Get all permno-ncusip combinations
_crsp1 = db.raw_sql("""
                      select permno, ncusip, comnam, namedt, nameenddt
                      from crsp.stocknames where ncusip != ''
                      """, date_cols=['namedt', 'nameenddt'])

In [10]:
_crsp1

,permno,ncusip,comnam,namedt,nameenddt
0,10000.0,68391610,OPTIMUM MANUFACTURING INC,1986-01-07,1987-06-11
1,10001.0,39040610,GREAT FALLS GAS CO,1986-01-09,1993-11-21
2,10001.0,29274A10,ENERGY WEST INC,1993-11-22,2008-02-04
3,10001.0,29274A20,ENERGY WEST INC,2008-02-05,2009-08-03
4,10001.0,29269V10,ENERGY INC,2009-08-04,2009-12-17
...,...,...,...,...,...
73588,93433.0,92870X30,VOLTARI CORP,2013-04-24,2016-12-22
73589,93434.0,78513510,S & W SEED CO,2010-06-14,2022-12-30
73590,93435.0,82936G20,SINO CLEAN ENERGY INC,2010-06-14,2012-05-18
73591,93436.0,88160R10,TESLA MOTORS INC,2010-06-29,2017-02-01


In [11]:
_crsp1_fnamedt = _crsp1.groupby(['permno','ncusip']).namedt.min().reset_index()

_crsp1_lnameenddt = _crsp1.groupby(['permno','ncusip']).nameenddt.max().reset_index()

# merge 
_crsp1_dtrange = pd.merge(_crsp1_fnamedt, _crsp1_lnameenddt, \
                          on = ['permno','ncusip'], how='inner')

_crsp1 = _crsp1.drop(['namedt'],axis=1).rename(columns={'nameenddt':'enddt'})
_crsp2 = pd.merge(_crsp1, _crsp1_dtrange, on =['permno','ncusip'], how='inner')

# keep only most recent company name
_crsp2 = _crsp2.loc[_crsp2.enddt ==_crsp2.nameenddt].drop(['enddt'], axis=1)

In [12]:
# 1.3 Create CUSIP Link Table

_link1_1 = pd.merge(_ibes2, _crsp2, how='inner', left_on='cusip', right_on='ncusip')\
.sort_values(['ticker','permno','ldate'])

_link1_1_tmp = _link1_1.groupby(['ticker','permno']).ldate.max().reset_index()
_link1_2 = pd.merge(_link1_1, _link1_1_tmp, how='inner', on =['ticker', 'permno', 'ldate'])


In [13]:
_link1_2

,ticker,cusip,cname,fdate,ldate,permno,ncusip,comnam,namedt,nameenddt
0,0000,87482X10,TALMER BANCORP,2014-02-20,2014-03-20,14471.0,87482X10,TALMER BANCORP INC,2014-02-12,2016-08-31
1,0001,26878510,EP ENGR CORP,2014-02-20,2019-10-17,14392.0,26878510,E P ENERGY CORP,2014-01-17,2019-05-22
2,0004,02504D10,AMERICAN CAPITAL,2014-02-20,2014-02-20,14418.0,02504D10,AMERICAN CAPITAL SR FLOATING LTD,2014-01-16,2018-08-24
3,000R,14163310,CARECOM,2014-02-20,2014-02-20,14378.0,14163310,CARE COM INC,2014-01-24,2020-02-10
4,000V,28249U10,EIGER,2016-04-14,2016-04-14,14423.0,28249U10,EIGER BIOPHARMACEUTICALS INC,2016-03-23,2022-12-30
...,...,...,...,...,...,...,...,...,...,...
21282,ZXIS,98876010,Z-AXIS,2004-06-17,2004-10-14,83970.0,98876010,Z AXIS CORP,1983-10-26,1985-11-01
21283,ZXZX,16951E10,CHINA ZENIX,2011-07-14,2018-07-19,12720.0,16951E10,CHINA ZENIX AUTO INTL LTD,2011-05-12,2018-06-13
21284,ZY,87254010,TJX,1989-07-20,2016-06-16,40539.0,87254010,T J X COMPANIES INC NEW,1989-06-21,2022-12-30
21285,ZYNE,98986X10,ZYNERBA PHARMS,2015-09-17,2015-09-17,15646.0,98986X10,ZYNERBA PHARMACEUTICALS INC,2015-08-05,2022-12-30


In [14]:
link_table = _link1_2[['permno', 'ncusip']]

### 2) Use Linking Table on IBES and CRSP files

In [15]:
IBES = pd.read_csv('data/ibes.csv')
CRSP = pd.read_csv('data/crsp.csv')

In [16]:
CRSP['rankdate']=pd.to_datetime(CRSP.date)
CRSP['rankdate'] = CRSP['rankdate'].dt.to_period('M')
CRSP['ret'] = pd.to_numeric(CRSP['ret'], errors='coerce')
CRSP = CRSP.sort_values(by=['permno','rankdate'], ascending=True)
CRSP.head()

,Unnamed: 0,permno,cusip,date,cfacshr,price,shrcd,exchcd,ret,rankdate
32,32,10000.0,68391610,1986-01-31,1.0,4.375000,10.0,3.0,NaN,1986-01
33,33,10000.0,68391610,1986-02-28,1.0,3.250000,10.0,3.0,-0.257143,1986-02
34,34,10000.0,68391610,1986-03-31,1.0,4.437500,10.0,3.0,0.365385,1986-03
35,35,10000.0,68391610,1986-04-30,1.0,4.000000,10.0,3.0,-0.098592,1986-04
36,36,10000.0,68391610,1986-05-30,1.0,3.109375,10.0,3.0,-0.222656,1986-05


In [17]:
IBES.head()

,Unnamed: 0,ticker,cusip,cname,fpedats,statpers,meanest,actual,anndats_act,fpi
0,0,0000,87482X10,TALMER BANCORP,2014-03-31,2014-04-17,0.08,0.12,2014-05-06,6
1,1,0000,87482X10,TALMER BANCORP,2014-06-30,2014-05-15,0.13,0.27,2014-08-06,6
2,2,0000,87482X10,TALMER BANCORP,2014-06-30,2014-06-19,0.13,0.27,2014-08-06,6
3,3,0000,87482X10,TALMER BANCORP,2014-06-30,2014-07-17,0.13,0.27,2014-08-06,6
4,4,0000,87482X10,TALMER BANCORP,2014-06-30,2014-04-17,0.12,0.27,2014-08-06,7


In [18]:
IBES_link = pd.merge(IBES, link_table, how='inner', left_on=['cusip'], right_on=['ncusip']).drop('ncusip', axis=1)
IBES_link.head()

,Unnamed: 0,ticker,cusip,cname,fpedats,statpers,meanest,actual,anndats_act,fpi,permno
0,0,0000,87482X10,TALMER BANCORP,2014-03-31,2014-04-17,0.08,0.12,2014-05-06,6,14471.0
1,1,0000,87482X10,TALMER BANCORP,2014-06-30,2014-05-15,0.13,0.27,2014-08-06,6,14471.0
2,2,0000,87482X10,TALMER BANCORP,2014-06-30,2014-06-19,0.13,0.27,2014-08-06,6,14471.0
3,3,0000,87482X10,TALMER BANCORP,2014-06-30,2014-07-17,0.13,0.27,2014-08-06,6,14471.0
4,4,0000,87482X10,TALMER BANCORP,2014-06-30,2014-04-17,0.12,0.27,2014-08-06,7,14471.0


In [19]:
IBES_link['rankdate'] = pd.to_datetime(IBES_link.statpers).dt.to_period('M')
IBES_link['permno'] = IBES_link['permno'].astype('int')
IBES_link.head()

,Unnamed: 0,ticker,cusip,cname,fpedats,statpers,meanest,actual,anndats_act,fpi,permno,rankdate
0,0,0000,87482X10,TALMER BANCORP,2014-03-31,2014-04-17,0.08,0.12,2014-05-06,6,14471,2014-04
1,1,0000,87482X10,TALMER BANCORP,2014-06-30,2014-05-15,0.13,0.27,2014-08-06,6,14471,2014-05
2,2,0000,87482X10,TALMER BANCORP,2014-06-30,2014-06-19,0.13,0.27,2014-08-06,6,14471,2014-06
3,3,0000,87482X10,TALMER BANCORP,2014-06-30,2014-07-17,0.13,0.27,2014-08-06,6,14471,2014-07
4,4,0000,87482X10,TALMER BANCORP,2014-06-30,2014-04-17,0.12,0.27,2014-08-06,7,14471,2014-04


In [20]:
IBES_CRSP = pd.merge(CRSP[['permno', 'price', 'ret', 'cfacshr', 'rankdate']], IBES_link, how='inner',  on=['permno', 'rankdate'])
IBES_CRSP.head()

,permno,price,ret,cfacshr,rankdate,Unnamed: 0,ticker,cusip,cname,fpedats,statpers,meanest,actual,anndats_act,fpi
0,10001.0,10.78,0.028992,1.0,2011-01,446864,GFGC,36720410,GAS NATURAL INC,2010-12-31,2011-01-20,1.05,0.92,2011-04-05,1
1,10001.0,10.78,0.028992,1.0,2011-01,446867,GFGC,36720410,GAS NATURAL INC,2010-12-31,2011-01-20,0.36,0.24,2011-04-05,6
2,10001.0,10.78,0.028992,1.0,2011-01,446872,GFGC,36720410,GAS NATURAL INC,2011-03-31,2011-01-20,0.47,0.52,2011-05-11,7
3,10001.0,10.78,0.028992,1.0,2011-01,446879,GFGC,36720410,GAS NATURAL INC,2011-06-30,2011-01-20,0.03,0.04,2011-08-12,8
4,10001.0,10.78,0.028992,1.0,2011-01,446901,GFGC,36720410,GAS NATURAL INC,2011-12-31,2011-01-20,0.73,0.58,2012-04-10,2


In [21]:
IBES_CRSP.to_csv('data/ibes_crsp.csv')

## Macroeconomic Data

In [26]:
GDP_Raw = pd.read_csv('data/real_GDP_FED.csv', index_col=0)
IPT_Raw = pd.read_csv('data/IPT_FED.csv', skiprows=range(1, 620), index_col=0)
IPT_Raw.drop(IPT_Raw.columns[1:121], axis=1, inplace=True)
IPT_Raw.reset_index(inplace=True, drop=True)
Cons_Raw = pd.read_csv('data/real_personal_consumption_FED.csv', index_col=0)
Unempl_Raw = pd.read_csv('data/Unemployment_FED.csv',skiprows=range(1, 225), index_col=0)

In [27]:
IPT_Raw

,DATE,IPT72M11,IPT72M12,IPT73M1,IPT73M2,IPT73M3,IPT73M4,IPT73M5,IPT73M6,IPT73M7,...,IPT23M3,IPT23M4,IPT23M5,IPT23M6,IPT23M7,IPT23M8,IPT23M9,IPT23M10,IPT23M11,IPT23M12
0,1970:08,107.5,107.5,107.5,107.5,107.5,107.5,107.5,107.5,107.5,...,37.7,37.6,37.6,37.6,37.6,37.6,37.6,37.6,37.6,37.6
1,1970:09,106.3,106.3,106.3,106.3,106.3,106.3,106.3,106.3,106.3,...,37.4,37.4,37.4,37.4,37.4,37.4,37.4,37.4,37.4,37.4
2,1970:10,103.7,103.7,103.7,103.7,103.7,103.7,103.7,103.7,103.7,...,36.7,36.6,36.6,36.6,36.6,36.6,36.6,36.6,36.6,36.6
3,1970:11,102.8,102.8,102.8,102.8,102.8,102.8,102.8,102.8,102.8,...,36.5,36.4,36.4,36.4,36.4,36.4,36.4,36.4,36.4,36.4
4,1970:12,104.9,104.9,104.9,104.9,104.9,104.9,104.9,104.9,104.9,...,37.3,37.2,37.2,37.2,37.2,37.2,37.2,37.2,37.2,37.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,2023:07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,102.9,103.1,103.3,103.3,103.2
636,2023:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,103.5,103.3,103.3,103.2
637,2023:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.6,103.4,103.3
638,2023:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.7,102.4


In [31]:
# Function to prepare data, except for Unemployment data
def PrepareMacro(Macro_Data,Begin_Year,Begin_Month,Name_col,Name_Var):
    
    #Initilising the data
    month = Begin_Month
    dates = []  #list to store dates
    values = [] #list to store values
    shape = Macro_Data.shape
    n_columns = shape[1]
    col = 1

    #Loop to extract the data
    for i in range(0,n_columns+1):
        year = (Begin_Year + i) % 100
    
        
        while month <= 12:
            if col == n_columns:
                break
            else:
                year_string = str("{:02d}".format(year))
                month_string = str(month)


                col_name = Name_col + year_string + 'M' + month_string
                A = Macro_Data[col_name]
                B = pd.value_counts(A.isna().values)
            
                if A.count() == shape[0]: #Check for when no NaNs
                    values.append(A.iloc[-1])
                else:
                    values.append(A[B.iloc[1]-1])

                if year >= Begin_Year:
                    dates.append('19'+year_string+'-'+month_string)
                else:
                    dates.append('20'+year_string+'-'+month_string)

                month += 1
                col += 1
        month = 1

    #Saving everything in a dataframe
    d = {'Dates':dates, Name_Var:values}
    y = pd.DataFrame(data=d)
    y['Dates'] = pd.to_datetime(y['Dates'], format='%Y-%m').dt.to_period('M')
    
    return y

GDP_Data = PrepareMacro(GDP_Raw,65,11,'ROUTPUT','GDP');
IPT_Data = PrepareMacro(IPT_Raw,72,11,'IPT','IPT');
Cons_Data = PrepareMacro(Cons_Raw,65,11,'RCON','Cons');

In [35]:
IPT_Data

,Dates,IPT
0,1972-11,116.7
1,1972-12,118.5
2,1973-01,119.3
3,1973-02,119.8
4,1973-03,120.8
...,...,...
609,2023-08,36.4
610,2023-09,36.6
611,2023-10,37.4
612,2023-11,37.6


In [36]:
#Different method for unemployemnt because data organised differently
Unempl_Arr = Unempl_Raw.to_numpy()
shape = Unempl_Arr.shape
N = shape[0]
values = []
for i in range(0,N):
    x = Unempl_Arr[i,1:]
    first_non_nan_value = next((value for value in x if not np.isnan(value)), None)
    values.append(first_non_nan_value)

#DataFrame
d = {'Dates':Unempl_Raw['DATE'], 'Unempl':values}
Unempl_Data = pd.DataFrame(data=d)
Unempl_Data['Dates'] = Unempl_Data['Dates'].str.replace(':', '-')
Unempl_Data['Dates'] = pd.to_datetime(Unempl_Data['Dates'], format='%Y-%m').dt.to_period('M')


In [37]:
Unempl_Data

,Dates,Unempl
224,1965-09,4.4
225,1965-10,4.3
226,1965-11,4.2
227,1965-12,4.1
228,1966-01,4.0
...,...,...
917,2023-06,3.6
918,2023-07,3.5
919,2023-08,3.8
920,2023-09,3.8


### Log of the differences (not for unemployment rate)

In [38]:
dfs_difference = [GDP_Data, Cons_Data, IPT_Data ]
cols_difference = ['GDP', 'Cons', 'IPT' ]
count = 0
for df, name in zip(dfs_difference, cols_difference):
    column_name = name + '_' + 'log_return'
    df[column_name] = np.log(df[name] / df[name].shift(1))
    df.dropna(inplace=True)


### Merge

In [39]:
merged_macro = Unempl_Data
for file in dfs_difference:
    merged_macro = pd.merge(merged_macro,file, on = ['Dates'], how = 'outer')

merged_macro.to_csv('data/processed_data/macro_data.csv')
merged_macro.head()

,Dates,Unempl,GDP,GDP_log_return,Cons,Cons_log_return,IPT,IPT_log_return
0,1965-09,4.4,NaN,NaN,NaN,NaN,NaN,NaN
1,1965-10,4.3,NaN,NaN,NaN,NaN,NaN,NaN
2,1965-11,4.2,NaN,NaN,NaN,NaN,NaN,NaN
3,1965-12,4.1,609.7,0.000985,396.7,0.016521,NaN,NaN
4,1966-01,4.0,609.7,0.000000,396.7,0.000000,NaN,NaN


## Financial Ratio

In [57]:
finratio = pd.read_csv('data/finratio.csv', index_col=0)
finratio.head()

/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/1677955836.py:1: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  finratio = pd.read_csv('data/finratio.csv', index_col=0)


,gvkey,permno,adate,qdate,public_date,capei,be,bm,evm,pe_op_basic,...,ffi30_desc,ffi30,ffi38_desc,ffi38,ffi48_desc,ffi48,ffi49_desc,ffi49,ticker,cusip
0,1001,10015,1983-12-31,1984-09-30,1985-01-31,NaN,9.105,0.551751,7.158486,NaN,...,MEALS,28.0,RTAIL,34.0,MEALS,43.0,MEALS,44.0,AMFD,00016510
1,1001,10015,1984-12-31,1984-12-31,1985-02-28,NaN,9.319,0.696487,5.660224,NaN,...,MEALS,28.0,RTAIL,34.0,MEALS,43.0,MEALS,44.0,AMFD,00016510
2,1001,10015,1984-12-31,1984-12-31,1985-03-31,NaN,9.319,0.696487,5.660224,NaN,...,MEALS,28.0,RTAIL,34.0,MEALS,43.0,MEALS,44.0,AMFD,00016510
3,1001,10015,1984-12-31,1984-12-31,1985-04-30,NaN,9.319,0.696487,5.660224,NaN,...,MEALS,28.0,RTAIL,34.0,MEALS,43.0,MEALS,44.0,AMFD,00016510
4,1001,10015,1984-12-31,1985-03-31,1985-05-31,NaN,9.630,0.491414,6.339642,NaN,...,MEALS,28.0,RTAIL,34.0,MEALS,43.0,MEALS,44.0,AMFD,00016510


In [58]:
print(finratio.columns.tolist())

['gvkey', 'permno', 'adate', 'qdate', 'public_date', 'capei', 'be', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf', 'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'gprof', 'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct', 'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets', 'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio', 'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn', 'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc', 'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ret_crsp', 'gsector', 'gicdesc', 'mktcap', 'price', 'ptb', 'peg_trailing', 'divyield', 'peg_1yrforward', 'peg_ltgforward', 'ffi5_desc', 'ffi5', 'ff

In [59]:
#We exclude PEG 1yrforward, PEG ltgforward, pe op basic, and pe op dil from our forecast model, because these variables have too many missing observations.
finratio.drop(['peg_1yrforward', 'peg_ltgforward', 'pe_op_basic', 'pe_op_dil', 'price', 'ret_crsp'], axis=1, inplace=True)

In [60]:
finratio['rankdate'] = pd.to_datetime(finratio['public_date'].astype('str')).dt.to_period('M')
finratio.drop(['gvkey', 'adate', 'qdate', 'public_date', 'ticker', 'cusip', 'ffi5_desc', 'ffi5', 'ffi10_desc', 'ffi10', 'ffi12_desc', 'ffi12', 'ffi17_desc', 'ffi17',
                'ffi30_desc', 'ffi30', 'ffi38_desc', 'ffi38', 'ffi48_desc', 'ffi48', 'ffi49_desc','gsector', 'gicdesc' ], axis=1, inplace=True)

In [61]:
vars_winsorize = list(finratio.drop(['permno'], axis=1).columns)
vars_winsorize

['capei',
 'be',
 'bm',
 'evm',
 'pe_exi',
 'pe_inc',
 'ps',
 'pcf',
 'dpr',
 'npm',
 'opmbd',
 'opmad',
 'gpm',
 'ptpm',
 'cfm',
 'roa',
 'roe',
 'roce',
 'efftax',
 'aftret_eq',
 'aftret_invcapx',
 'aftret_equity',
 'pretret_noa',
 'pretret_earnat',
 'gprof',
 'equity_invcap',
 'debt_invcap',
 'totdebt_invcap',
 'capital_ratio',
 'int_debt',
 'int_totdebt',
 'cash_lt',
 'invt_act',
 'rect_act',
 'debt_at',
 'debt_ebitda',
 'short_debt',
 'curr_debt',
 'lt_debt',
 'profit_lct',
 'ocf_lct',
 'cash_debt',
 'fcf_ocf',
 'lt_ppent',
 'dltt_be',
 'debt_assets',
 'debt_capital',
 'de_ratio',
 'intcov',
 'intcov_ratio',
 'cash_ratio',
 'quick_ratio',
 'curr_ratio',
 'cash_conversion',
 'inv_turn',
 'at_turn',
 'rect_turn',
 'pay_turn',
 'sale_invcap',
 'sale_equity',
 'sale_nwc',
 'rd_sale',
 'adv_sale',
 'staff_sale',
 'accrual',
 'mktcap',
 'ptb',
 'peg_trailing',
 'divyield',
 'ffi49',
 'rankdate']

In [62]:
finratio = finratio.dropna(axis=0, subset=['ffi49'])

In [63]:
# Replace NA with the industry median
finratio.loc[:, vars_winsorize] = finratio.groupby(['rankdate', 'ffi49'])[vars_winsorize].transform(lambda x: x.fillna(x.median(skipna=True)))
print(finratio[vars_winsorize].isna().sum().sum())

# Replace remaining NA values with last month's value
finratio.loc[:, vars_winsorize] = finratio.groupby('permno')[vars_winsorize].transform(lambda x: x.ffill().bfill())
print(finratio[vars_winsorize].isna().sum().sum())

# Replace any remaining NA with the industry median again
finratio.loc[:, vars_winsorize] = finratio.groupby(['rankdate', 'ffi49'])[vars_winsorize].transform(lambda x: x.fillna(x.median(skipna=True)))
print(finratio[vars_winsorize].isna().sum().sum())


/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/4102184784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finratio.loc[:, vars_winsorize] = finratio.groupby(['rankdate', 'ffi49'])[vars_winsorize].transform(lambda x: x.fillna(x.median(skipna=True)))
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/4102184784.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  finratio.loc[:, vars_winsorize] = finratio.groupby(['rankdate', 'ffi49'])[vars_winsorize].transform(lambda x: x.fillna(x.median(skipna=

1500


/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/4102184784.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finratio.loc[:, vars_winsorize] = finratio.groupby('permno')[vars_winsorize].transform(lambda x: x.ffill().bfill())
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/4102184784.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  finratio.loc[:, vars_winsorize] = finratio.groupby('permno')[vars_winsorize].transform(lambda x: x.ffill().bfill())


0


/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/4102184784.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finratio.loc[:, vars_winsorize] = finratio.groupby(['rankdate', 'ffi49'])[vars_winsorize].transform(lambda x: x.fillna(x.median(skipna=True)))
/var/folders/gf/n_mlnzkx2hq9v2tbywpcl_mm0000gn/T/ipykernel_19409/4102184784.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  finratio.loc[:, vars_winsorize] = finratio.groupby(['rankdate', 'ffi49'])[vars_winsorize].transform(lambda x: x.fillna(x.median(skipn

0


In [64]:
for column in finratio.columns:
    na_count = finratio[column].isna().sum()
    if na_count > 0:
        print(f"{column}: {na_count}")

### Merge FinRatio with IBES_CRSP

In [66]:
IBES_CRSP.dtypes

permno           float64
price            float64
ret              float64
cfacshr          float64
rankdate       period[M]
Unnamed: 0         int64
ticker            object
cusip             object
cname             object
fpedats           object
statpers          object
meanest          float64
actual           float64
anndats_act       object
fpi                int64
dtype: object

In [67]:
data = pd.merge(IBES_CRSP, finratio, how='inner', on=['permno', 'rankdate'])

In [68]:
print(data.shape)
data.head()

(5309783, 85)


,permno,price,ret,cfacshr,rankdate,Unnamed: 0,ticker,cusip,cname,fpedats,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,mktcap,ptb,peg_trailing,divyield,ffi49
0,10001.0,10.78,0.028992,1.0,2011-01,446864,GFGC,36720410,GAS NATURAL INC,2010-12-31,...,10.349008,0.0,0.0,0.0,0.02788,84.450518,1.55086,0.053007,0.050093,31.0
1,10001.0,10.78,0.028992,1.0,2011-01,446867,GFGC,36720410,GAS NATURAL INC,2010-12-31,...,10.349008,0.0,0.0,0.0,0.02788,84.450518,1.55086,0.053007,0.050093,31.0
2,10001.0,10.78,0.028992,1.0,2011-01,446872,GFGC,36720410,GAS NATURAL INC,2011-03-31,...,10.349008,0.0,0.0,0.0,0.02788,84.450518,1.55086,0.053007,0.050093,31.0
3,10001.0,10.78,0.028992,1.0,2011-01,446879,GFGC,36720410,GAS NATURAL INC,2011-06-30,...,10.349008,0.0,0.0,0.0,0.02788,84.450518,1.55086,0.053007,0.050093,31.0
4,10001.0,10.78,0.028992,1.0,2011-01,446901,GFGC,36720410,GAS NATURAL INC,2011-12-31,...,10.349008,0.0,0.0,0.0,0.02788,84.450518,1.55086,0.053007,0.050093,31.0


## FINAL: DATASETS SEPARTED BY FPI

In [69]:
A1 = data[data.fpi == 1].reset_index(drop=True)
A2= data[data.fpi == 2].reset_index(drop=True)
Q1 = data[data.fpi == 6].reset_index(drop=True)
Q2 = data[data.fpi == 7].reset_index(drop=True)
Q3 = data[data.fpi == 8].reset_index(drop=True)

In [70]:
#adjust with cfacshr
for Forecast in [A1, A2, Q1, Q2, Q3]:
    df_name = next(name for name, obj in globals().items() if obj is Forecast)
    Forecast['adj_actual'] = Forecast.groupby('permno', group_keys=True).apply(lambda x: x['actual'] * x['cfacshr'].shift(1) / x['cfacshr']).values
    Forecast['adj_meanest'] = Forecast.groupby('permno', group_keys=True).apply(lambda x: x['meanest'] * x['cfacshr'].shift(1) / x['cfacshr']).values
    Forecast.dropna(subset=['adj_actual', 'meanest'], inplace=True)
    Forecast.sort_values(by=['permno', 'rankdate'], ascending=True, inplace=True)
    Forecast.reset_index(drop=True, inplace=True)
    Forecast.to_csv(f"data/processed_data/{df_name}.csv", index=False)